### Ici, « Candidat », au sens initial, peut désigner chaque fragment d’entité identifié par le modèle,tandis qu’« entité » fait plutôt référence à l’entité nommée complète, après agrégation.



“candidat” 在原始意义上可以指模型识别的 每个实体片段，而“entité”更倾向于表示 完整聚合后的命名实体。

In [ ]:
import json
from transformers import pipeline, CamembertTokenizer, TokenClassificationPipeline

# === Initialisation du modèle CamemBERT spécialisé pour la reconnaissance d'entités nommées (NER) ===
model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = CamembertTokenizer.from_pretrained(model_name)

# Création d'un pipeline NER avec agrégation simple (fusion des tokens associés à la même entité)
nlp: TokenClassificationPipeline = pipeline(
    "ner",
    model=model_name,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # Agrège les sous-tokens appartenant à une même entité
    device=-1  # -1 = CPU, 0 = GPU (si disponible)
)

# === Texte d'exemple à analyser ===
texte = "Je teste spacy à Paris"

# === Tokenisation du texte : transformation en sous-unités exploitables par le modèle ===
tokens = tokenizer.tokenize(texte)

# === Application du pipeline de reconnaissance d'entités nommées ===
ner_results = nlp(texte)

# === Formatage des entités reconnues sous forme de dictionnaire simple (mot + type) ===
liste_entitesnom = [
    {"mot": ent["word"], "type": ent["entity_group"]}
    for ent in ner_results
]

# === Affichage des résultats ===
print(f"Tokens        : {len(tokens)}")               # Nombre total de tokens (sous-mots)
print(f"Candidats     : {len(ner_results)}")          # Nombre d'entités reconnues par le modèle
print("-" * 20)
print(f"Ex_tokens     : {tokens}")                    # Affichage des 20 premiers tokens (ou moins)
print(f"Ex_entités    : {liste_entitesnom}")          # Liste des entités identifiées avec leur type
print(f"Ex_candidats  : {ner_results}")               # Affichage des 20 premiers candidats (ou moins)




Device set to use cpu


Tokens        : 6
Candidats     : 2
--------------------
Ex_tokens     : ['▁Je', '▁teste', '▁spa', 'cy', '▁à', '▁Paris']
Ex_entités    : [{'mot': 'spacy', 'type': 'MISC'}, {'mot': 'Paris', 'type': 'LOC'}]
Ex_candidats  : [{'entity_group': 'MISC', 'score': np.float32(0.96689326), 'word': 'spacy', 'start': None, 'end': None}, {'entity_group': 'LOC', 'score': np.float32(0.9827497), 'word': 'Paris', 'start': None, 'end': None}]


### Exemple de REN pour un ouvrage dans nôtre corpus

https://github.com/These-SCAI2023/CORPUS/blob/master/prog/Use_camemBERT.py

In [ ]:
from transformers import pipeline, TokenClassificationPipeline, CamembertTokenizer
from itertools import chain

# === Initialisation du modèle et du tokenizer CamemBERT ===
model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
nlp: TokenClassificationPipeline = pipeline(
    "ner",
    model=model_name,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # Agrégation des sous-tokens en entités complètes
    device=-1  # -1 pour utiliser le CPU, 0 pour GPU si disponible
)

# === Lecture du fichier texte ===
file_path = "sample_data/Corpus/APPOLINAIRE_Caligrammes.txt"
with open(file_path, encoding="utf-8") as f:
    texte = f.read()

# === Fonction pour diviser le texte en morceaux (chunks) avec chevauchement ===
def chunk_text(text, chunk_size, overlap):
    start = 0
    while start < len(text):
        end = start + chunk_size
        yield text[start:end]
        start += chunk_size - overlap

# === Conversion d'une entité brute en dictionnaire structuré ===
def get_entity_dict(entity: dict) -> dict:
    return {
        "label": entity['entity_group'],      # Type d'entité (PER, LOC, etc.)
        "text": entity['word'],               # Texte de l'entité
        "jalons": [entity['start'], entity['end']]  # Indices de position dans le texte
    }

# === Traitement du texte avec le modèle NER pour extraire les entités ===
def dico_resultats(texte, nlp: TokenClassificationPipeline, chunk_size=512, overlap=50):
    chunks = list(chunk_text(texte, chunk_size, overlap))  # Diviser le texte
    all_ner_results = list(chain.from_iterable([nlp(chunk) for chunk in chunks]))  # Appliquer le NER sur chaque chunk
    entity_list = [get_entity_dict(entity) for entity in all_ner_results]  # Formatter chaque entité
    return entity_list

# === Exécution de la reconnaissance d'entités nommées ===
liste_entitesnom = dico_resultats(texte, nlp)

# === Affichage des informations utiles ===
tokens = tokenizer.tokenize(texte)  # Tokenisation du texte (subword level)
print(f"Tokens        : {len(tokens)}")  # Nombre total de tokens générés
print(f"Candidats     : {len(liste_entitesnom)}")  # Nombre d'entités détectées
print("-" * 20)
print(f"Ex_tokens     : {tokens[:20]}")  # Affichage des 20 premiers tokens
ex_entites = [{"mot": ent["text"], "type": ent["label"]} for ent in liste_entitesnom[:20]]  # Format simplifié des entités
print(f"Ex_entités    : {ex_entites}")  # Affichage des 20 premières entités extraites




Device set to use cpu


Tokens        : 21295
Candidats     : 732
--------------------
Ex_tokens     : ['▁Rappel', '▁de', '▁votre', '▁demande', ':', '▁Format', '▁de', '▁téléchargement', ':', '▁:', '▁Texte', '▁Vue', 's', '▁1', '▁à', '▁26', '2', '▁sur', '▁26', '2']
Ex_entités    : [{'mot': 'Texte Vues', 'type': 'MISC'}, {'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'de La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': '(1880-1918', 'type': 'DATE'}, {'mot': '(Lausanne)', 'type': 'LOC'}, {'mot': '1952', 'type': 'DATE'}, {'mot': 'La Fresnaye, Roger de', 'type': 'PER'}, {'mot': '(1885-1925', 'type': 'DATE'}, {'mot': 'bnf.fr', 'type': 'LOC'}, {'mot': 'bnf', 'type': 'LOC'}, {'mot': 'atalogue', 'type': 'MISC'}, {'mot': 'bnf', 'type': 'ORG'}, {'mot': 'Français', 'type': 'MISC'}, {'mot': 'in', 'type': 'DATE'}, {'mot': '[Calligrammes', 'type': 'MISC'}, {'mot': 'Collection', 'type': 'MISC'}, {'mot': 'Collect

In [ ]:
import json
result = {
    "entites": liste_entitesnom
}
with open("tous_entites_nommes_Camembert.json", "w", encoding="utf-8") as json_file:
    json.dump(result, json_file, indent=4, ensure_ascii=False)

print("Résultats enregistrés dans 'tous_entites_nommes_Camembert.json'")


Résultats enregistrés dans 'tous_entites_nommes_Camembert.json'


### Ttaiter au niveau de chaque texte

In [ ]:
import glob
import time
import os
import json
from transformers import pipeline, TokenClassificationPipeline, CamembertTokenizer
from itertools import chain
from tqdm import tqdm

# === Initialisation du modèle CamemBERT ===
model_name = "Jean-Baptiste/camembert-ner-with-dates"
tokenizer = CamembertTokenizer.from_pretrained(model_name)
nlp: TokenClassificationPipeline = pipeline(
    "ner",
    model=model_name,
    tokenizer=tokenizer,
    aggregation_strategy="simple",
    device=-1  # CPU
)

# === Fonction : diviser le texte en morceaux exploitables par le modèle ===
def chunk_text(text, chunk_size=512, overlap=50):
    start = 0
    while start < len(text):
        end = start + chunk_size
        yield text[start:end]
        start += chunk_size - overlap

# === Fonction : transformer une entité en format structuré ===
def get_entity_dict(entity: dict) -> dict:
    return {
        "label": entity['entity_group'],
        "text": entity['word'],
        "jalons": [entity['start'], entity['end']]
    }

# === Fonction : exécuter le NER sur un texte donné ===
def dico_resultats(texte, nlp: TokenClassificationPipeline, chunk_size=512, overlap=50):
    chunks = list(chunk_text(texte, chunk_size, overlap))
    all_ner_results = list(chain.from_iterable([nlp(chunk) for chunk in chunks]))
    return [get_entity_dict(entity) for entity in all_ner_results]

# === Créer un dossier pour les fichiers JSON (si non existant) ===
os.makedirs("output_json", exist_ok=True)

# === Traitement de chaque fichier texte ===
for path_fichier in glob.glob("sample_data/Corpus/*.txt"):
    file_name = os.path.basename(path_fichier)
    print(f"Traitement du fichier : {file_name}")
    start_time = time.time()

    with open(path_fichier, "r", encoding="utf-8") as f:
        texte = f.read().strip()

    # Extraction des entités
    liste_entitesnom = dico_resultats(texte, nlp)

    # Tokenisation
    tokens = tokenizer.tokenize(texte)

    # === Affichage dans la console ===
    print(f"Tokens        : {len(tokens)}")
    print(f"Candidats     : {len(liste_entitesnom)}")
    print("-" * 20)
    print(f"Ex_tokens     : {tokens[:20]}")
    ex_entites = [{"mot": ent["text"], "type": ent["label"]} for ent in liste_entitesnom[:20]]
    print(f"Ex_entités    : {ex_entites}")
    print("=" * 40)
    print(f"Temps de traitement : {time.time() - start_time:.2f} secondes")

    # === Sauvegarde en JSON : uniquement {mot, type} ===
    entites_simplifiees = [{"mot": ent["text"], "type": ent["label"]} for ent in liste_entitesnom]
    result = {"entites": entites_simplifiees}

    nom_fichier_json = file_name.replace(".txt", "_entites.json")
    output_path = f"sample_data/output_json/{nom_fichier_json}"

    with open(output_path, "w", encoding="utf-8") as f_json:
        json.dump(result, f_json, indent=4, ensure_ascii=False)




Device set to use cpu


Traitement du fichier : sample_data/Corpus/SAUVAGE_Tandis-que-la-terre-tourne.txt
Tokens        : 26772
Candidats     : 245
--------------------
Ex_tokens     : ['▁Rappel', '▁de', '▁votre', '▁demande', ':', '▁Format', '▁de', '▁téléchargement', ':', '▁:', '▁Texte', '▁Vue', 's', '▁1', '▁à', '▁20', '4', '▁sur', '▁20', '4']
Ex_entités    : [{'mot': 'Texte Vues', 'type': 'MISC'}, {'mot': 'Tandis que la terre tourne', 'type': 'MISC'}, {'mot': 'Cécile Sauvage', 'type': 'PER'}, {'mot': 'Sauvage, Cécile', 'type': 'PER'}, {'mot': '(1883-1927', 'type': 'DATE'}, {'mot': 'Mercure de France', 'type': 'ORG'}, {'mot': 'Paris', 'type': 'LOC'}, {'mot': '1910', 'type': 'DATE'}, {'mot': 'nf', 'type': 'ORG'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': '28/01/2020', 'type': 'DATE'}, {'mot': 'CÉCILE SA', 'type': 'PER'}, {'mot': 'UV', 'type': 'MISC'}, {'mot': 'AGE', 'type': 'PER'}, {'mot': '’andis', 'type': 'MISC'}, {'mot': '